In [ ]:
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, BatchNormalization, Dropout,LSTM
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Attention,
                                     Flatten, Reshape, UpSampling3D, MaxPool3D, AveragePooling2D)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import TimeDistributed, GRU, InputLayer, Softmax
from tensorflow.keras import Model
early_stopping = EarlyStopping(patience=30)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

## data

In [ ]:
import pickle
with open('D:\\py\\deeppy\\sst_data4.pickle','rb') as file:
    sst_data = pickle.load(file)

In [ ]:
minidata = sst_data[:,200:400,250:450]

In [ ]:
minval = np.min(np.where(minidata>0,minidata,5000))
maxval = np.max(minidata)

In [ ]:
minidata = (minidata+0.000001-minval)/(maxval+0.000001-minval)

In [ ]:
minidata = np.where(minidata>0,minidata,0)

In [ ]:
west = sst_data[:,275:285,301:311]
east = sst_data[:,275:285,396:406]
south = sst_data[:,322:332,349:359]

In [ ]:
eastdiff = np.max(east)-np.min(east)
eastnor = (east-np.min(east))/(np.max(east) - np.min(east))

In [ ]:
diff = (maxval+0.000001-minval)

In [ ]:
testdata = minidata[:,150:160,40:50]

## Conv3D+LSTM

### tuning

In [ ]:
def conv3d(filter1,filter_size, lstm, day):
    adam = tf.keras.optimizers.Adam(lr=0.001, decay=0.1)
    inputs = Input((day,10,10,1))
    output = Conv3D(filter1,filter_size,padding='same')(inputs)
    output = MaxPool3D(2)(output)
    output = Conv3D(int(filter1*(day/16)),filter_size,padding='same')(output)
    output = MaxPool3D(2)(output)
    output = Reshape((day,int(filter1/4)))(output)
    output = LSTM(lstm, activation='relu',return_sequences=True)(output)
    output = Flatten()(output)
    output = Dense(100)(output)
    output = Reshape((10,10))(output)
    model2 = Model(inputs, output)
    model2.compile(loss='mse', optimizer=adam)
    model2.summary()
    
    return model2

In [ ]:
filter1 = map(int,[128,256,512])
filter_size=[3,5,7,9]
lstm=[30,45,60]
day = map(int,[5,7,9])
result=[]

In [ ]:
for l in day:    
    test = testdata[:,np.newaxis,:,:]
    for m in range(l-1):
        test = np.append(test[:-1], testdata[(m+1):,np.newaxis], axis=1)
    for i in filter1:
        for j in filter_size:
            for k in lstm:
                model_t = conv3d(i,j,k,l)
                history = model_t.fit(test[:-366], testdata[(l):-365], batch_size=200, epochs=2000,  validation_split=0.03)
                result.append(np.sum(abs(model_t.predict(test[-366:-1])-testdata[-365:].reshape((365,10,10)))*diff)/(100*365))
                print(l,i,j,k)
                tf.keras.backend.clear_session()

In [ ]:
# 5일치  sst data
test5 = testdata[:,np.newaxis,:,:]
for i in range(4):
    test5 = np.append(test5[:-1], testdata[(i+1):,np.newaxis], axis=1)

In [ ]:
adam = tf.keras.optimizers.Adam(lr=0.001, decay=0.1)
inputs = Input((5,10,10,1))
output = Conv3D(256,5,padding='same')(inputs)
output = MaxPool3D(2)(output)
output = Conv3D(120,5,padding='same')(output)
output = MaxPool3D(2)(output)
output = Reshape((5,96))(output)
output = LSTM(60, activation='relu',return_sequences=True)(output)
output = Flatten()(output)
output = Dense(100)(output)
output = Reshape((10,10))(output)
model1 = Model(inputs, output)
model1.compile(loss='mse', optimizer=adam)
model1.summary()

In [ ]:
h = model1.fit(test14[:-366], testdata[5:-365], batch_size=200, epochs=2000,  validation_split=0.03)

In [1]:
# last 365 days are test set
np.sum(abs(model1.predict(test5[-366:-1])-testdata[-365:].reshape((365,10,10)))*diff)/(100*365)

NameError: name 'np' is not defined

## Conv3D+LSTM+Attention

In [ ]:
adam = tf.keras.optimizers.Adam(lr=0.001, decay=0.1)
inputs = Input(shape=(5,10,10,1))
cnn3d = Conv3D(256,5,padding='same')(inputs)
cnn3d = MaxPool3D(2)(cnn3d)
cnn3d = BatchNormalization()(cnn3d)
cnn3d = Conv3D(120,5,padding='same')(cnn3d)
cnn3d = MaxPool3D(2)(cnn3d)
cnn3d = Reshape((5,96))(cnn3d)

lstm = LSTM(60, return_sequences=True)(cnn3d)
attention = Attention()([lstm,lstm])
sent = Flatten()(attention)
output = Dense(100)(sent)
output = Reshape((10,10))(output)
model = Model(inputs, output)
model.compile(loss='mse', optimizer=adam)
model.summary()

In [ ]:
at=model.fit(test5[:-366], testdata[5:-365], batch_size=100, epochs=2000,  validation_split=0.03)

In [ ]:
np.sum(abs(model.predict(test5[-366:-1])-testdata[-365:].reshape((365,10,10)))*diff)/(100*365) #test

## ConvLSTM

In [ ]:
adam = tf.keras.optimizers.Adam(lr=0.001, decay=0.1)
inputs = Input((5,10,10,1))
output = ConvLSTM2D(128,5,return_sequences=True,padding='same')(inputs)
output = ConvLSTM2D(64,5, padding='same')(output)
output = Flatten()(output)
output = Dense(100)(output)
output = Reshape((10,10))(output)
modelcl2 = Model(inputs, output)
modelcl2.compile(loss='mse', optimizer=adam)
modelcl2.summary()

In [ ]:
cl2=modelcl2.fit(test14[:-366], testdata[5:-365], batch_size=100, epochs=3000,  validation_split=0.03)

In [ ]:
np.sum(abs(modelcl2.predict(test5[-366:-1])-testdata[-365:].reshape((365,10,10)))*diff)/(100*365) #test